In [4]:
!nvidia-smi

Wed Jun 16 17:22:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# !pip install torch
# !python -m pip install git+https://github.com/nicolas-chaulet/torch-points3d.git

In [ ]:
# !mkdir -p torch_points3d/datasets/segmentation

In [ ]:
# !pip3 install torch torchvision torchaudio

In [7]:
import torch
torch.cuda.is_available()


True

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!wget --no-check-certificate https://shapenet.cs.stanford.edu/media/shapenetcore_partanno_segmentation_benchmark_v0_normal.zip

--2021-06-16 17:19:59--  https://shapenet.cs.stanford.edu/media/shapenetcore_partanno_segmentation_benchmark_v0_normal.zip
Resolving shapenet.cs.stanford.edu (shapenet.cs.stanford.edu)... 171.67.77.19
Connecting to shapenet.cs.stanford.edu (shapenet.cs.stanford.edu)|171.67.77.19|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 200 OK
Length: 706988960 (674M) [application/zip]
Saving to: ‘shapenetcore_partanno_segmentation_benchmark_v0_normal.zip’

shapenetcore_partan 100%[===================>] 674.24M  16.5MB/s    in 68s     

2021-06-16 17:21:07 (9.92 MB/s) - ‘shapenetcore_partanno_segmentation_benchmark_v0_normal.zip’ saved [706988960/706988960]



In [2]:
!git clone https://github.com/qq456cvb/Point-Transformers.git

Cloning into 'Point-Transformers'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 69 (delta 23), reused 52 (delta 10), pack-reused 0
Unpacking objects: 100% (69/69), done.


In [ ]:
!unzip shapenetcore_partanno_segmentation_benchmark_v0_normal.zip
%cd /content/Point-Transformers
%mkdir data/
!mv /content/shapenetcore_partanno_segmentation_benchmark_v0_normal data/shapenetcore_partanno_segmentation_benchmark_v0_normal


In [22]:
# !cp -r /content/Point-Transformers/data/shapenetcore_partanno_segmentation_benchmark_v0_normal /content/drive/MyDrive/Datasets/Shapenet

cp: cannot stat '/content/Point-Transformers/data/shapenetcore_partanno_segmentation_benchmark_v0_normal': No such file or directory


In [22]:
!pip install hydra-core --upgrade
!pip install plyfile

Requirement already up-to-date: hydra-core in /usr/local/lib/python3.7/dist-packages (1.1.0)


In [9]:
!cp -r '/content/drive/MyDrive/OpenEDS 2021 challenge/val' /content/Point-Transformers/data

In [24]:
%%writefile /content/Point-Transformers/dataset.py
import numpy as np
import os
from torch.utils.data import Dataset
import torch
from pointnet_util import farthest_point_sample, pc_normalize
import json
import os.path as osp
from plyfile import PlyData, PlyElement

class ModelNetDataLoader(Dataset):
    def __init__(self, root, npoint=1024, split='train', uniform=False, normal_channel=True, cache_size=15000):
        self.root = root
        self.npoints = npoint
        self.uniform = uniform
        self.catfile = os.path.join(self.root, 'modelnet40_shape_names.txt')

        self.cat = [line.rstrip() for line in open(self.catfile)]
        self.classes = dict(zip(self.cat, range(len(self.cat))))
        self.normal_channel = normal_channel

        shape_ids = {}
        shape_ids['train'] = [line.rstrip() for line in open(os.path.join(self.root, 'modelnet40_train.txt'))]
        shape_ids['test'] = [line.rstrip() for line in open(os.path.join(self.root, 'modelnet40_test.txt'))]

        assert (split == 'train' or split == 'test')
        shape_names = ['_'.join(x.split('_')[0:-1]) for x in shape_ids[split]]
        # list of (shape_name, shape_txt_file_path) tuple
        self.datapath = [(shape_names[i], os.path.join(self.root, shape_names[i], shape_ids[split][i]) + '.txt') for i
                         in range(len(shape_ids[split]))]
        print('The size of %s data is %d'%(split,len(self.datapath)))

        self.cache_size = cache_size  # how many data points to cache in memory
        self.cache = {}  # from index to (point_set, cls) tuple

    def __len__(self):
        return len(self.datapath)

    def _get_item(self, index):
        if index in self.cache:
            point_set, cls = self.cache[index]
        else:
            fn = self.datapath[index]
            cls = self.classes[self.datapath[index][0]]
            cls = np.array([cls]).astype(np.int32)
            point_set = np.loadtxt(fn[1], delimiter=',').astype(np.float32)
            if self.uniform:
                point_set = farthest_point_sample(point_set, self.npoints)
            else:
                point_set = point_set[0:self.npoints,:]

            point_set[:, 0:3] = pc_normalize(point_set[:, 0:3])

            if not self.normal_channel:
                point_set = point_set[:, 0:3]

            if len(self.cache) < self.cache_size:
                self.cache[index] = (point_set, cls)

        return point_set, cls

    def __getitem__(self, index):
        return self._get_item(index)


class PartNormalDataset(Dataset):
    def __init__(self, root='./data/shapenetcore_partanno_segmentation_benchmark_v0_normal', npoints=2500, split='train', class_choice=None, normal_channel=False):
        self.npoints = npoints
        # self.root = root
        # self.catfile = os.path.join(self.root, 'synsetoffset2category.txt')
        self.cat = {}
        self.normal_channel = normal_channel
        self.folder = "val"
        self.data_dir = os.path.join(root, self.folder)

        self.datapath = []
        for file in os.listdir(self.data_dir):   
            if os.path.isdir(osp.join(self.data_dir,file)):
                self.datapath.append(file)
            else:
                print("wrong file path, please reconsider your life choices")
                raise IOError 

        # self.classes = {}
        # for i in self.cat.keys():
        #     self.classes[i] = self.classes_original[i]

        # print(self.classes)
        # print(self.datapath)
        # Mapping from category ('Chair') to a list of int [10,11,12,13] as segmentation labels
        self.seg_classes = {'Pupil' :[0], 'Iris': [1], 'Sclera': [2], 'Eye-lashes': [3], 'Background':[4]}

        # for cat in sorted(self.seg_classes.keys()):
        #     print(cat, self.seg_classes[cat])

        self.cache = {}  # from index to (point_set, cls, seg) tuple
        self.cache_size = 20000


    def __getitem__(self, index):
        if index in self.cache:
            point_set, seg = self.cache[index]
        else:
            fn = self.datapath[index]
            data = self.load_point_cloud(fn)
            point_set = data[:, 0:3]
            labels_path = osp.join(self.data_dir, fn, "labels.npy")
            seg = np.load(labels_path).astype(np.int32) 
            # seg = data[:, -1].astype(np.int32)
            if len(self.cache) < self.cache_size:
                self.cache[index] = (point_set, seg)
        point_set[:, 0:3] = pc_normalize(point_set[:, 0:3])

        choice = np.random.choice(len(seg), self.npoints, replace=True)
        # resample
        point_set = point_set[choice, :]
        seg = seg[choice]

        return point_set, seg

    def load_point_cloud(self,filepath):
        pointCloudPath = osp.join(self.data_dir,filepath,"pointcloud.ply")
        
        plydata = PlyData.read(pointCloudPath)
        return np.array(np.transpose(np.stack((plydata['vertex']['x'],plydata['vertex']['y'],plydata['vertex']['z'])))).astype(np.float32)


    def __len__(self):
        return len(self.datapath)



        
    #     if download and not os.path.exists(self.data_dir):
    #         print("folder not found, please check dataset")

    #     self.train, self.num_points = train, num_points

    #     self.points, self.labels = [],[]
    #     for file in os.listdir(self.data_dir):   
    #         if os.path.isdir(osp.join(self.data_dir,file)):
    #             # print(os.listdir(osp.join(self.filepath,file)))
    #             pointCloudPath = osp.join(self.data_dir,file,"pointcloud.ply")
        
    #             plydata = PlyData.read(pointCloudPath)
    #             pts = np.array(np.transpose(np.stack((plydata['vertex']['x'],plydata['vertex']['y'],plydata['vertex']['z'])))).astype(np.float32)

    #             choice = np.random.choice(len(pts), 4096, replace=True)
    #             point_set = pts[choice, :]

    #             point_set = point_set - np.expand_dims(np.mean(point_set, axis=0), 0)  # center
    #             dist = np.max(np.sqrt(np.sum(point_set ** 2, axis=1)), 0)
    #             point_set = point_set / dist  # scale


    #             self.points.append(point_set)


    #             labels_path = osp.join(self.data_dir, file, "labels.npy")
    #             label = np.load(labels_path)
    #             label = label[choice]
    #             self.labels.append(label)

    #     self.points = np.stack(self.points)
    #     self.labels = np.stack(self.labels)



    #     # print(self.points.shape, self.labels.shape)
    # def __getitem__(self, idx):
    #     pt_idxs = np.arange(0, self.num_points)
    #     np.random.shuffle(pt_idxs)

    #     current_points = torch.from_numpy(self.points[idx, pt_idxs].copy()).float()
    #     current_labels = torch.from_numpy(self.labels[idx, pt_idxs].copy()).long()
    #     return current_points, current_labels

    # def __len__(self):
    #     return int(self.points.shape[0] * self.data_precent)

    # def set_num_points(self, pts):
    #     self.num_points = pts


if __name__ == '__main__':
    data = PartNormalDataset('/content/Point-Transformers/data', split='train')
    DataLoader = torch.utils.data.DataLoader(data, batch_size=12, shuffle=True)
    for point,label in DataLoader:
        print(point.shape)
        print(label.shape)

Overwriting /content/Point-Transformers/dataset.py


In [25]:
!python /content/Point-Transformers/dataset.py

torch.Size([12, 2500, 3])
torch.Size([12, 2500])
torch.Size([12, 2500, 3])
torch.Size([12, 2500])
torch.Size([12, 2500, 3])
torch.Size([12, 2500])
torch.Size([12, 2500, 3])
torch.Size([12, 2500])
torch.Size([12, 2500, 3])
torch.Size([12, 2500])
torch.Size([12, 2500, 3])
torch.Size([12, 2500])
torch.Size([6, 2500, 3])
torch.Size([6, 2500])


In [11]:
!ls

config	dataset.py  pointnet_util.py  __pycache__  train_cls.py
data	models	    provider.py       README.md    train_partseg.py


In [28]:
%%writefile /content/Point-Transformers/train_partseg.py
"""
Author: Benny
Date: Nov 2019
"""
import argparse
import os
import torch
import datetime
import logging
import sys
import importlib
import shutil
import provider
import numpy as np

from pathlib import Path
from tqdm import tqdm
from dataset import PartNormalDataset
import hydra
import omegaconf


# seg_classes = {'Earphone': [16, 17, 18], 'Motorbike': [30, 31, 32, 33, 34, 35], 'Rocket': [41, 42, 43],
#                'Car': [8, 9, 10, 11], 'Laptop': [28, 29], 'Cap': [6, 7], 'Skateboard': [44, 45, 46], 'Mug': [36, 37],
#                'Guitar': [19, 20, 21], 'Bag': [4, 5], 'Lamp': [24, 25, 26, 27], 'Table': [47, 48, 49],
#                'Airplane': [0, 1, 2, 3], 'Pistol': [38, 39, 40], 'Chair': [12, 13, 14, 15], 'Knife': [22, 23]}

seg_classes = {'Pupil' :[0], 'Iris': [1], 'Sclera': [2], 'Eye-lashes' :[3], 'Background':[4]}
seg_label_to_cat = {}  # {0:Airplane, 1:Airplane, ...49:Table}
for cat in seg_classes.keys():
    for label in seg_classes[cat]:
        seg_label_to_cat[label] = cat


def inplace_relu(m):
    classname = m.__class__.__name__
    if classname.find('ReLU') != -1:
        m.inplace=True

def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    new_y = torch.eye(num_classes)[y.cpu().data.numpy(),]
    if (y.is_cuda):
        return new_y.cuda()
    return new_y

@hydra.main(config_path='config', config_name='partseg')
def main(args):
    omegaconf.OmegaConf.set_struct(args, False)

    '''HYPER PARAMETER'''
    # os.environ["CUDA_VISIBLE_DEVICES"] = str(args.gpu)
    logger = logging.getLogger(__name__)

    # print(args.pretty())

    root = hydra.utils.to_absolute_path('data/')

    TRAIN_DATASET = PartNormalDataset(root=root, npoints=args.num_point, split='trainval', normal_channel=args.normal)
    trainDataLoader = torch.utils.data.DataLoader(TRAIN_DATASET, batch_size=args.batch_size, shuffle=True, num_workers=10, drop_last=True)
    TEST_DATASET = PartNormalDataset(root=root, npoints=args.num_point, split='test', normal_channel=args.normal)
    testDataLoader = torch.utils.data.DataLoader(TEST_DATASET, batch_size=args.batch_size, shuffle=False, num_workers=10)

    '''MODEL LOADING'''
    # args.input_dim = (6 if args.normal else 3) + 16
    args.input_dim = (3 if args.normal else 3)
    args.num_class = 5
    num_category = 5
    num_part = args.num_class
    shutil.copy(hydra.utils.to_absolute_path('models/{}/model.py'.format(args.model.name)), '.')

    classifier = getattr(importlib.import_module('models.{}.model'.format(args.model.name)), 'PointTransformerSeg')(args).cuda()
    criterion = torch.nn.CrossEntropyLoss()

    try:
        checkpoint = torch.load('best_model.pth')
        start_epoch = checkpoint['epoch']
        classifier.load_state_dict(checkpoint['model_state_dict'])
        logger.info('Use pretrain model')
    except:
        logger.info('No existing model, starting training from scratch...')
        start_epoch = 0

    if args.optimizer == 'Adam':
        optimizer = torch.optim.Adam(
            classifier.parameters(),
            lr=args.learning_rate,
            betas=(0.9, 0.999),
            eps=1e-08,
            weight_decay=args.weight_decay
        )
    else:
        optimizer = torch.optim.SGD(classifier.parameters(), lr=args.learning_rate, momentum=0.9)

    def bn_momentum_adjust(m, momentum):
        if isinstance(m, torch.nn.BatchNorm2d) or isinstance(m, torch.nn.BatchNorm1d):
            m.momentum = momentum

    LEARNING_RATE_CLIP = 1e-5
    MOMENTUM_ORIGINAL = 0.1
    MOMENTUM_DECCAY = 0.5
    MOMENTUM_DECCAY_STEP = args.step_size

    best_acc = 0
    global_epoch = 0
    best_class_avg_iou = 0
    best_inctance_avg_iou = 0

    for epoch in range(start_epoch, args.epoch):
        mean_correct = []

        logger.info('Epoch %d (%d/%s):' % (global_epoch + 1, epoch + 1, args.epoch))
        '''Adjust learning rate and BN momentum'''
        lr = max(args.learning_rate * (args.lr_decay ** (epoch // args.step_size)), LEARNING_RATE_CLIP)
        logger.info('Learning rate:%f' % lr)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        momentum = MOMENTUM_ORIGINAL * (MOMENTUM_DECCAY ** (epoch // MOMENTUM_DECCAY_STEP))
        if momentum < 0.01:
            momentum = 0.01
        print('BN momentum updated to: %f' % momentum)
        classifier = classifier.apply(lambda x: bn_momentum_adjust(x, momentum))
        classifier = classifier.train()

        '''learning one epoch'''
        for i, (points,  target) in tqdm(enumerate(trainDataLoader), total=len(trainDataLoader), smoothing=0.9):
            points = points.data.numpy()
            points[:, :, 0:3] = provider.random_scale_point_cloud(points[:, :, 0:3])
            points[:, :, 0:3] = provider.shift_point_cloud(points[:, :, 0:3])
            points = torch.Tensor(points)
            # k = torch.cat([points, to_categorical(label, num_category).repeat(1, points.shape[1], 1)], -1)
            # print(points.shape, label, target.shape,k.shape)
            points,  target = points.float().cuda(), target.long().cuda()
            optimizer.zero_grad()
            seg_pred = classifier(points)
            # seg_pred = classifier(torch.cat([points, to_categorical(label, num_category).repeat(1, points.shape[1], 1)], -1))
            # print(seg_pred.shape, target[:,10])
            seg_pred = seg_pred.contiguous().view(-1, num_part)
            target = target.view(-1, 1)[:, 0]
            pred_choice = seg_pred.data.max(1)[1]

            correct = pred_choice.eq(target.data).cpu().sum()
            mean_correct.append(correct.item() / (args.batch_size * args.num_point))
            loss = criterion(seg_pred, target)
            loss.backward()
            optimizer.step()

        train_instance_acc = np.mean(mean_correct)
        logger.info('Train accuracy is: %.5f' % train_instance_acc)

        with torch.no_grad():
            test_metrics = {}
            total_correct = 0
            total_seen = 0
            total_seen_class = [0 for _ in range(num_part)]
            total_correct_class = [0 for _ in range(num_part)]
            shape_ious = {cat: [] for cat in seg_classes.keys()}
            seg_label_to_cat = {}  # {0:Airplane, 1:Airplane, ...49:Table}

            for cat in seg_classes.keys():
                for label in seg_classes[cat]:
                    seg_label_to_cat[label] = cat

            classifier = classifier.eval()

            for batch_id, (points, label, target) in tqdm(enumerate(testDataLoader), total=len(testDataLoader), smoothing=0.9):
                cur_batch_size, NUM_POINT, _ = points.size()
                points, label, target = points.float().cuda(), label.long().cuda(), target.long().cuda()
                seg_pred = classifier(torch.cat([points, to_categorical(label, num_category).repeat(1, points.shape[1], 1)], -1))
                cur_pred_val = seg_pred.cpu().data.numpy()
                cur_pred_val_logits = cur_pred_val
                cur_pred_val = np.zeros((cur_batch_size, NUM_POINT)).astype(np.int32)
                target = target.cpu().data.numpy()

                for i in range(cur_batch_size):
                    cat = seg_label_to_cat[target[i, 0]]
                    logits = cur_pred_val_logits[i, :, :]
                    cur_pred_val[i, :] = np.argmax(logits[:, seg_classes[cat]], 1) + seg_classes[cat][0]

                correct = np.sum(cur_pred_val == target)
                total_correct += correct
                total_seen += (cur_batch_size * NUM_POINT)

                for l in range(num_part):
                    total_seen_class[l] += np.sum(target == l)
                    total_correct_class[l] += (np.sum((cur_pred_val == l) & (target == l)))

                for i in range(cur_batch_size):
                    segp = cur_pred_val[i, :]
                    segl = target[i, :]
                    cat = seg_label_to_cat[segl[0]]
                    part_ious = [0.0 for _ in range(len(seg_classes[cat]))]
                    for l in seg_classes[cat]:
                        if (np.sum(segl == l) == 0) and (
                                np.sum(segp == l) == 0):  # part is not present, no prediction as well
                            part_ious[l - seg_classes[cat][0]] = 1.0
                        else:
                            part_ious[l - seg_classes[cat][0]] = np.sum((segl == l) & (segp == l)) / float(
                                np.sum((segl == l) | (segp == l)))
                    shape_ious[cat].append(np.mean(part_ious))

            all_shape_ious = []
            for cat in shape_ious.keys():
                for iou in shape_ious[cat]:
                    all_shape_ious.append(iou)
                shape_ious[cat] = np.mean(shape_ious[cat])
            mean_shape_ious = np.mean(list(shape_ious.values()))
            test_metrics['accuracy'] = total_correct / float(total_seen)
            test_metrics['class_avg_accuracy'] = np.mean(
                np.array(total_correct_class) / np.array(total_seen_class, dtype=np.float))
            for cat in sorted(shape_ious.keys()):
                logger.info('eval mIoU of %s %f' % (cat + ' ' * (14 - len(cat)), shape_ious[cat]))
            test_metrics['class_avg_iou'] = mean_shape_ious
            test_metrics['inctance_avg_iou'] = np.mean(all_shape_ious)

        logger.info('Epoch %d test Accuracy: %f  Class avg mIOU: %f   Inctance avg mIOU: %f' % (
            epoch + 1, test_metrics['accuracy'], test_metrics['class_avg_iou'], test_metrics['inctance_avg_iou']))
        if (test_metrics['inctance_avg_iou'] >= best_inctance_avg_iou):
            logger.info('Save model...')
            savepath = 'best_model.pth'
            logger.info('Saving at %s' % savepath)
            state = {
                'epoch': epoch,
                'train_acc': train_instance_acc,
                'test_acc': test_metrics['accuracy'],
                'class_avg_iou': test_metrics['class_avg_iou'],
                'inctance_avg_iou': test_metrics['inctance_avg_iou'],
                'model_state_dict': classifier.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }
            torch.save(state, savepath)
            logger.info('Saving model....')

        if test_metrics['accuracy'] > best_acc:
            best_acc = test_metrics['accuracy']
        if test_metrics['class_avg_iou'] > best_class_avg_iou:
            best_class_avg_iou = test_metrics['class_avg_iou']
        if test_metrics['inctance_avg_iou'] > best_inctance_avg_iou:
            best_inctance_avg_iou = test_metrics['inctance_avg_iou']
        logger.info('Best accuracy is: %.5f' % best_acc)
        logger.info('Best class avg mIOU is: %.5f' % best_class_avg_iou)
        logger.info('Best inctance avg mIOU is: %.5f' % best_inctance_avg_iou)
        global_epoch += 1


if __name__ == '__main__':
    main()

Overwriting /content/Point-Transformers/train_partseg.py


In [34]:
%%writefile /content/Point-Transformers/config/partseg.yaml
batch_size: 12
epoch: 200
learning_rate: 1e-3
gpu: 1
num_point: 1024
optimizer: Adam
weight_decay: 1e-4
normal: False
lr_decay: 0.5
step_size: 20

defaults:
  - model: Hengshuang

hydra:
  run:
    dir: log/partseg/${model.name}

  sweep:
    dir: log/partseg
    subdir: ${model.name}

Overwriting /content/Point-Transformers/config/partseg.yaml


In [14]:
%cd /content/Point-Transformers
%ls

/content/Point-Transformers
config/  dataset.py  pointnet_util.py  __pycache__/  train_cls.py
data/    models/     provider.py       README.md     train_partseg.py


In [35]:
!python /content/Point-Transformers/train_partseg.py

/usr/local/lib/python3.7/dist-packages/hydra/core/default_element.py:127: UserWarning: In 'model/Hengshuang': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  See {url} for more information"""
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
[2021-06-16 18:14:45,591][__main__][INFO] - No existing model, starting training from scratch...
[2021-06-16 18:14:45,592][__main__][INFO] - Epoch 1 (1/200):
[2021-06-16 18:14:45,592][__main__][INFO] - Learning rate:0.001000
BN mom

## Part 2: Point transformers using S3DIS



In [28]:
!nvidia-smi

Wed Jun 16 14:53:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    70W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# !wget --no-check-certificat https://shapenet.cs.stanford.edu/media/indoor3d_sem_seg_hdf5_data.zip

--2021-06-14 07:03:11--  https://shapenet.cs.stanford.edu/media/indoor3d_sem_seg_hdf5_data.zip
Resolving shapenet.cs.stanford.edu (shapenet.cs.stanford.edu)... 171.67.77.19
Connecting to shapenet.cs.stanford.edu (shapenet.cs.stanford.edu)|171.67.77.19|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 200 OK
Length: 1679781976 (1.6G) [application/zip]
Saving to: ‘indoor3d_sem_seg_hdf5_data.zip’

indoor3d_sem_seg_hd 100%[===================>]   1.56G  1.31MB/s    in 10m 57s 

2021-06-14 07:14:08 (2.44 MB/s) - ‘indoor3d_sem_seg_hdf5_data.zip’ saved [1679781976/1679781976]



In [ ]:
# !unzip /content/indoor3d_sem_seg_hdf5_data.zip

In [ ]:
# import h5py
# def load_data_file(name):
#     f = h5py.File(name, "r")
#     data = f["data"][:]
#     label = f["label"][:]
#     return data, label

In [ ]:
# d, l = load_data_file('/content/Point-Transformers/indoor3d_sem_seg_hdf5_data/ply_data_all_0.h5')
# print(d.shape, l.shape)

(1000, 4096, 9) (1000, 4096)


In [ ]:
# %cd ..

/content


In [2]:
!git clone https://github.com/POSTECH-CVLab/point-transformer.git

Cloning into 'point-transformer'...
remote: Enumerating objects: 341, done.
remote: Counting objects: 100% (341/341), done.
remote: Compressing objects: 100% (238/238), done.
remote: Total 341 (delta 183), reused 229 (delta 93), pack-reused 0
Receiving objects: 100% (341/341), 78.57 KiB | 924.00 KiB/s, done.
Resolving deltas: 100% (183/183), done.


In [3]:
%cd point-transformer/
%ls
!pip install -r requirements.txt

/content/point-transformer
LICENSE.md   point_transformer/      pyproject.toml  requirements.txt  tox.ini
MANIFEST.in  point_transformer_lib/  README.rst      setup.py
Processing ./point_transformer_lib
     |████████████████████████████████| 81kB 3.8MB/s 
     |████████████████████████████████| 6.0MB 7.3MB/s 
     |████████████████████████████████| 188.4MB 88kB/s 
     |████████████████████████████████| 829kB 21.3MB/s 
  Created wheel for pytorch-lightning: filename=pytorch_lightning-0.7.1-cp37-none-any.whl size=145330 sha256=f7b0695d1c02da853fc7c8a08a942f7169ad6023f77f2d77b954c932929f152e
  Stored in directory: /root/.cache/pip/wheels/dc/93/61/14094d2116ff739513dda993007501ae5701b78386b39d5912
  Created wheel for point-transformer-ops: filename=point_transformer_ops-0.1.0-cp37-cp37m-linux_x86_64.whl size=5415562 sha256=4b64d9d7fef76b925ba4ce32ec75e763766c41a63ebb3d3e3bc7c22575b81c60
  Stored in directory: /tmp/pip-ephem-wheel-cache-r69uet5m/wheels/f7/62/54/9c2001c27dfd0e9a91559fbd41a

In [4]:
!pip install -e .

Obtaining file:///content/point-transformer
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Running setup.py develop for point-transformer


In [10]:
!mkdir /content/point-transformer/point_transformer/data/val

In [11]:
!cp -r '/content/drive/MyDrive/OpenEDS 2021 challenge/val' /content/point-transformer/point_transformer/data/val

In [8]:
%cd /content/point-transformer

/content/point-transformer


In [ ]:
# %%writefile /content/point-transformer/point_transformer/config/config.yaml
# defaults:
#     - task: cls
#     - model: point_transformer
#     - task_model: ${defaults.0.task}-${defaults.1.model}

# hydra:
#   run:
#     dir: outputs

# gpus:
#     - 0

# optimizer: ???

# task_model: ???

# model: ???

# distrib_backend: dp

Overwriting /content/point-transformer/point_transformer/config/config.yaml


In [ ]:
%%writefile /content/point-transformer/point_transformer/data/Indoor3DSemSegLoader.py

import os
import shlex
import subprocess
from plyfile import PlyData, PlyElement
import h5py
import numpy as np
import torch
import torch.utils.data as data
import os.path as osp

BASE_DIR = os.path.dirname(os.path.abspath(__file__))


def _get_data_files(list_filename):
    with open(list_filename) as f:
        return [line.rstrip() for line in f]


def _load_data_file(name):
    f = h5py.File(name, "r")
    data = f["data"][:]
    label = f["label"][:]
    return data, label


class Indoor3DSemSeg(data.Dataset):
    def __init__(self, num_points, train=True, download=False, data_precent=1.0):
        super().__init__()
        self.data_precent = data_precent
        self.folder = "val/val"
        self.data_dir = os.path.join(BASE_DIR, self.folder)
        self.url = (
            "https://shapenet.cs.stanford.edu/media/indoor3d_sem_seg_hdf5_data.zip"
        )

        if download and not os.path.exists(self.data_dir):
            # zipfile = os.path.join(BASE_DIR, os.path.basename(self.url))
            # subprocess.check_call(
            #     shlex.split("curl {} -o {}".format(self.url, zipfile))
            # )

            # subprocess.check_call(
            #     shlex.split("unzip {} -d {}".format(zipfile, BASE_DIR))
            # )

            # subprocess.check_call(shlex.split("rm {}".format(zipfile)))
            print("folder not found, please check dataset")

        self.train, self.num_points = train, num_points

        # all_files = _get_data_files(os.path.join(self.data_dir, "all_files.txt"))
        # room_filelist = _get_data_files(
        #     os.path.join(self.data_dir, "room_filelist.txt")
        # )

        # data_batchlist, label_batchlist = [], []
        # for f in all_files:
        #     data, label = _load_data_file(os.path.join(BASE_DIR, f))
        #     data_batchlist.append(data)
        #     label_batchlist.append(label)

        # data_batches = np.concatenate(data_batchlist, 0)
        # labels_batches = np.concatenate(label_batchlist, 0)

        # test_area = "Area_5"
        # train_idxs, test_idxs = [], []
        # for i, room_name in enumerate(room_filelist):
        #     if test_area in room_name:
        #         test_idxs.append(i)
        #     else:
        #         train_idxs.append(i)

        # if self.train:
        #     self.points = data_batches[train_idxs, ...]
        #     self.labels = labels_batches[train_idxs, ...]
        # else:
        #     self.points = data_batches[test_idxs, ...]
        #     self.labels = labels_batches[test_idxs, ...]

        self.points, self.labels = [],[]
        for file in os.listdir(self.data_dir):   
            if os.path.isdir(osp.join(self.data_dir,file)):
                # print(os.listdir(osp.join(self.filepath,file)))
                pointCloudPath = osp.join(self.data_dir,file,"pointcloud.ply")
        
                plydata = PlyData.read(pointCloudPath)
                pts = np.array(np.transpose(np.stack((plydata['vertex']['x'],plydata['vertex']['y'],plydata['vertex']['z'])))).astype(np.float32)

                choice = np.random.choice(len(pts), 4096, replace=True)
                point_set = pts[choice, :]

                point_set = point_set - np.expand_dims(np.mean(point_set, axis=0), 0)  # center
                dist = np.max(np.sqrt(np.sum(point_set ** 2, axis=1)), 0)
                point_set = point_set / dist  # scale


                self.points.append(point_set)


                labels_path = osp.join(self.data_dir, file, "labels.npy")
                label = np.load(labels_path)
                label = label[choice]
                self.labels.append(label)

        self.points = np.stack(self.points)
        self.labels = np.stack(self.labels)



        # print(self.points.shape, self.labels.shape)
    def __getitem__(self, idx):
        pt_idxs = np.arange(0, self.num_points)
        np.random.shuffle(pt_idxs)

        current_points = torch.from_numpy(self.points[idx, pt_idxs].copy()).float()
        current_labels = torch.from_numpy(self.labels[idx, pt_idxs].copy()).long()
        return current_points, current_labels

    def __len__(self):
        return int(self.points.shape[0] * self.data_precent)

    def set_num_points(self, pts):
        self.num_points = pts

    def randomize(self):
        pass


if __name__ == "__main__":
    dset = Indoor3DSemSeg(16)
    print(dset[0])
    print(len(dset))
    dloader = torch.utils.data.DataLoader(dset, batch_size=32, shuffle=True)
    for i, data in enumerate(dloader, 0):
        inputs, labels = data
        if i == len(dloader) - 1:
            print(inputs.size())

Overwriting /content/point-transformer/point_transformer/data/Indoor3DSemSegLoader.py


In [9]:
!python -m point_transformer.train task=partseg batch_size=8

[2021-06-16 14:11:05,575][root][INFO] - GPU available: True, used: True
[2021-06-16 14:11:05,576][root][INFO] - VISIBLE GPUS: 0
Traceback (most recent call last):
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/content/point-transformer/point_transformer/train.py", line 59, in <module>
    main()
  File "/usr/local/lib/python3.7/dist-packages/hydra/main.py", line 24, in decorated_main
    strict=strict,
  File "/usr/local/lib/python3.7/dist-packages/hydra/_internal/utils.py", line 174, in run_hydra
    overrides=args.overrides,
  File "/usr/local/lib/python3.7/dist-packages/hydra/_internal/hydra.py", line 86, in run
    job_subdir_key=None,
  File "/usr/local/lib/python3.7/dist-packages/hydra/plugins/common/utils.py", line 109, in run_job
    ret.return_value = task_function(task_cfg)
  File "/content/point-transformer/point_transformer/train

In [12]:
import os
import shlex
import subprocess
from plyfile import PlyData, PlyElement
import h5py
import numpy as np
import torch
import torch.utils.data as data
import os.path as osp

data_dir = '/content/point-transformer/point_transformer/data/val/val'
points, labels = [],[]
for file in os.listdir('/content/point-transformer/point_transformer/data/val/val'):   
    if os.path.isdir(osp.join(data_dir,file)):
        # print(os.listdir(osp.join(self.filepath,file)))
        pointCloudPath = osp.join(data_dir,file,"pointcloud.ply")

        plydata = PlyData.read(pointCloudPath)
        pts = np.array(np.transpose(np.stack((plydata['vertex']['x'],plydata['vertex']['y'],plydata['vertex']['z'])))).astype(np.float32)

        choice = np.random.choice(len(pts), 4096, replace=True)
        point_set = pts[choice, :]

        point_set = point_set - np.expand_dims(np.mean(point_set, axis=0), 0)  # center
        dist = np.max(np.sqrt(np.sum(point_set ** 2, axis=1)), 0)
        point_set = point_set / dist  # scale


        points.append(point_set)
        

        labels_path = osp.join(data_dir, file, "labels.npy")
        label = np.load(labels_path)
        label = label[choice]
        labels.append(label)

In [13]:
ots = np.stack(labels)
ots.shape

(78, 4096)